In [ ]:
import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI
# from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [ ]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

In [ ]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama

llm_orca_mini = Ollama(
    model="orca-mini", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

### Load data

In [1]:
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("/Users/yinchangli/guardian/guardian/data/")

docs = loader.load()

docs_list = [doc.page_content for doc in docs]

In [2]:
from langchain.document_loaders import DirectoryLoader

### load code repo

In [ ]:
repo_path = "/Users/yinchangli/guardian/guardian/data/"

In [6]:
# from git import Repo
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import Language

In [9]:
# Load
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".py",".csv"],
    parser=LanguageParser()
)
documents = loader.load()
len(documents)

56

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)
texts = python_splitter.split_documents(documents)
len(texts)

### Split data

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

### Embedings

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings

embedding=OpenAIEmbeddings()
# model_name = 'sentence-transformers/all-MiniLM-L6-v2'
# model_name = 'intfloat/e5-large-v2'
# embedding_sentence_trans = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
# embedding_e5_large_v2 = HuggingFaceBgeEmbeddings(model_name = 'intfloat/e5-large-v2')

### Elastisearch

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import ElasticsearchStore

In [ ]:
es_db = ElasticsearchStore.from_documents(
    texts, 
    embedding, 
    es_url="http://localhost:9200", 
    index_name="index_godaddy",
    distance_strategy="COSINE",
)


In [ ]:
es_db = ElasticsearchStore.from_documents(
    texts, 
    embedding, 
    es_url="http://localhost:9200", 
    index_name="index_fod",
    distance_strategy="COSINE",
)


In [ ]:
es_db = ElasticsearchStore.from_documents(
    texts, 
    embedding, 
    es_url="http://localhost:9200", 
    index_name="index_ui_wb",
    distance_strategy="COSINE",
)


In [ ]:
es_db = ElasticsearchStore.from_documents(
    texts, 
    embedding, 
    es_url="http://localhost:9200", 
    index_name="index_repo",
    distance_strategy="COSINE",
)


In [ ]:
es_db = ElasticsearchStore.from_documents(
    docs, 
    embedding, 
    es_url="http://localhost:9200", 
    index_name="index_oai",
    distance_strategy="COSINE",
)


In [ ]:
es_db_e5 = ElasticsearchStore.from_documents(
    docs, 
    embedding_e5_large_v2, 
    es_url="http://localhost:9200", 
    index_name="index_e5",
    distance_strategy="COSINE",
)


In [ ]:
es_db_stn_trns = ElasticsearchStore.from_documents(
    docs, 
    embedding_sentence_trans, 
    es_url="http://localhost:9200", 
    index_name="inext_stn_trns",
    distance_strategy="COSINE",
)

In [ ]:
# es_db.client.indices.refresh(index="test")

# query = "How long does it take VA to make a decision?"
# results = es_db.similarity_search(query)
# print(results)

In [ ]:
es_retriever = es_db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

### Chroma Retriever

In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=texts, embedding=embedding)
chroma_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

### FAISS Retriever

In [ ]:
from langchain.vectorstores import FAISS

In [ ]:
faiss_vectorstore = FAISS.from_documents(docs, embedding)

In [ ]:
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 4})

### Self-querying

In [ ]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="disability_type",
        description="The type of disability being referenced. Examples include 'PTSD', 'hearing loss', 'musculoskeletal', 'mental health', etc.",
        type="string",
    ),
    AttributeInfo(
        name="compensation_criteria",
        description="Key criteria or conditions for compensation eligibility related to the disability",
        type="string",
    ),
    AttributeInfo(
        name="benefit_rate",
        description="The rate or range of VA disability compensation for the specific disability, often dependent on severity and other factors",
        type="string",
    ),
    AttributeInfo(
        name="documentation_required",
        description="Description of the documentation required for the compensation claim, such as medical records, service records, etc.",
        type="string",
    ),
]
document_content_description = "Answers to common questions or explanations related to VA disability compensation"


In [ ]:
self_retriever_chrome = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)

self_retriever_es = SelfQueryRetriever.from_llm(
    llm,
    es_db,
    document_content_description,
    metadata_field_info,
)


### Ensemble Retriever

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
bm25_retriever = BM25Retriever.from_texts(docs_list)
bm25_retriever.k = 2

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

### LLM Inference

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.memory import ConversationSummaryMemory

In [ ]:
def get_response(llm, retriever,query):
    qa = ConversationalRetrievalChain.from_llm(
        llm,
        retriever,
        return_source_documents=True
    )
    result = qa({"question": query, "chat_history":[]})
    return result['answer'], result['source_documents']
    

In [ ]:
# query = "what are the factors for determing the time it will take my claim?"
# query = "How long does it take VA to make a decision?"
query = 'where can i change open api?'

In [ ]:
response = get_response(llm, es_retriever, query)

In [ ]:
response[0]

In [ ]:
get_response(llm, chroma_retriever, query)

In [ ]:
get_response(llm_orca_mini, self_retriever_chrome, query)

In [ ]:
get_response(llm, faiss_retriever, query)

### Guardrails

In [ ]:
from nemoguardrails import LLMRails, RailsConfig

In [ ]:
# !pwd
CONFIG_PATH = "/Users/yinchangli/guardian/guardian/Topical_Rail/"

In [ ]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0)
config = RailsConfig.from_path(CONFIG_PATH)

In [ ]:
def get_guardrail_response(llm, retriever,query):
    qa = ConversationalRetrievalChain.from_llm(
        llm,
        retriever,
        return_source_documents=True
    )
    app = LLMRails(config = config, llm = llm)
    app.register_action(qa, name = 'conversation')
    history = [
    {"role": "user", "content":query}
]
    result = app.generate(messages = history)
    return result

In [ ]:
query = "How long does it take VA to make a decision?"
get_guardrail_response(llm, chroma_retriever, query)

In [ ]:
get_guardrail_response(llm_orca_mini, chroma_retriever, query)

## RAG System Evaluation

In [ ]:
import pandas as pd

df_test = pd.read_csv('questions_w_groundtruth.csv')

In [ ]:
answers = []
contexts = []
for question in df_test['question']:
    answer, context = get_response(llm,self_retriever_es, question)
    print('question: ' + question)
    print('answer: ' + answer)
    print('-----------------------------')
    answers.append(answer)
    contexts.append(context)

# Assign the results back to the DataFrame
df_test['answer'] = answers
df_test['contexts'] = contexts

In [ ]:
df_test

#### create evaluation dataset

In [ ]:
from datasets import Dataset, Features, Value, Sequence

In [ ]:
df = df_test

In [ ]:
data_dict = df.to_dict(orient = 'list')
df['ground_truths'] = df['ground_truths'].apply(lambda x: [x] if isinstance(x, str) else x)

In [ ]:
features = Features({
    'question': Value('string'),
    'ground_truths': Value('string'),
    'answer': Value('string'),
    'contexts': Sequence(Value('string'))
})

dataset = Dataset.from_dict(data_dict, features = features)

In [ ]:
dataset

In [ ]:
llm_eval = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)

In [ ]:
embedding=OpenAIEmbeddings()

In [ ]:
from ragas.metrics import AnswerCorrectness
from ragas.metrics import AnswerRelevancy
from ragas.metrics import ContextPrecision
from ragas.metrics import ContextRecall

answer_correctness = AnswerCorrectness()
answer_relevancy = AnswerRelevancy(
    embeddings=embedding
)

context_precision = ContextPrecision()
context_recall = ContextRecall()


In [ ]:
ans_correctness_score = answer_correctness.score(dataset)
df['answer_correctness'] = ans_correctness_score['answer_correctness']

In [ ]:
answer_relevancy_scores = answer_relevancy.score(dataset)
df['answer_relevancy'] = answer_relevancy_scores['answer_relevancy']

In [ ]:
ContextPrecision_scores = context_precision.score(dataset)
df['context_precision'] = ContextPrecision_scores['context_precision']

In [ ]:
context_recall_scores = context_recall.score(dataset)
df['context_recall'] = context_recall_scores['context_recall']

In [ ]:
df.to_csv('eval_result_openai.csv')